In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
from skimage import io
import seaborn as sns
import warnings
import numpy as np
import warnings
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pylab import mpl, plt
import matplotlib.patches as mpatches
from tqdm.notebook import tqdm

# best font and style settings for notebook 
warnings.filterwarnings('ignore')
sns.set_style("white")
mpl.rcParams['font.family'] = 'MiSans'

model_path = r"./Qwen3-0.6B"  # modify to your Qwen Path
# model_path = r"./Qwen3-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from delta_trainer import train_delta_from_H, generate_by_H, evaluate_slot_ceval, evaluate_slot_ceval_eos, \
    evaluate_slot_ceval_eos_2

# 构造 prompt & 得到 H_state
prompt = "请写一段关于AI教育的引言。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True, return_dict=True)
H = outputs.hidden_states[-1]

# 调用 delta 训练
delta_3 = train_delta_from_H(model, tokenizer, prompt, H, step=3)
delta_10 = train_delta_from_H(model, tokenizer, prompt, H, step=10)
delta_30 = train_delta_from_H(model, tokenizer, prompt, H, step=30)


In [ ]:
# generate_by_H(model=model, prompt=prompt, tokenizer=tokenizer, delta=delta_3, answer_len=200)

In [ ]:
from datasets import get_dataset_config_names

# 获取本地路径 "./ceval-exam" 中可用的所有子数据集名称（config names）
dataset_path = "./ceval-exam"
dataset_names = get_dataset_config_names(path=dataset_path)
dataset_names

In [ ]:
from datasets import load_dataset

dataset = load_dataset(r"./ceval-exam", name="computer_network")
print(dataset['val'][10])

In [ ]:
def evaluate_slot_ceval_eos(model, tokenizer, delta, example, max_len=20, verbose=True):
    """
    基于 generate_by_H_eos 的评估函数，用于 C-Eval 单选题目。

    返回：
    - predict_option: 预测选项，如 'A'
    - is_correct: 是否预测正确
    """
    prompt = f"""以下是一道单项选择题，请你阅读题目并选择最合适的选项。

题目：{example['question']}

选项：
A. {example['A']}
B. {example['B']}
C. {example['C']}
D. {example['D']}

答案是："""

    output_text = generate_by_H_eos(model, prompt, tokenizer, delta, answer_len=max_len)

    if verbose:
        print("🔍 模型生成结果:\n", output_text)

    predict_option = None
    for option in ['A', 'B', 'C', 'D']:
        if option in output_text:
            predict_option = option
            break

    is_correct = (predict_option == example['answer'])
    # return predict_option, is_correct
    return output_text, predict_option, example['answer'], is_correct

In [ ]:
from delta_trainer import generate_by_H_eos

import torch
from tqdm.notebook import tqdm


def generate_by_H_eos_fast(model, prompt, tokenizer, delta, answer_len=100):
    """
    使用 past_key_values 加速，支持 eos 截断的 H 层扰动生成。

    参数：
    - model: 支持 use_cache 的 decoder-only 模型（如 GPT 系列）
    - prompt: 输入文本
    - tokenizer: 分词器
    - delta: shape=[1, 1, hidden_size] 的扰动张量
    - answer_len: 最多生成 token 数

    返回：
    - record_txt: 解码后的文本（不含 prompt 部分）
    """
    eos_token_id = tokenizer.eos_token_id
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_ids = inputs["input_ids"]  # [1, L_prompt]

    with torch.no_grad():
        # 初始化推理，缓存 key_values
        outputs = model(input_ids=input_ids, return_dict=True, output_hidden_states=True, use_cache=True)
        past_key_values = outputs.past_key_values

        # 首个扰动 + 生成
        H_last = outputs.hidden_states[-1][:, -1, :] + delta.squeeze(1)  # [1, hidden_size]
        logits = torch.matmul(H_last, model.lm_head.weight.T)
        next_token_id = torch.argmax(logits, dim=-1, keepdim=True)  # [1, 1]

    record = [next_token_id]  # 收集生成 token

    for _ in range(answer_len - 1):  # 已生成 1 个，最多生成 answer_len 个
        if next_token_id.item() == eos_token_id:
            break

        with torch.no_grad():
            outputs = model(
                input_ids=next_token_id,
                past_key_values=past_key_values,
                return_dict=True,
                output_hidden_states=True,
                use_cache=True
            )
            past_key_values = outputs.past_key_values
            H_last = outputs.hidden_states[-1][:, -1, :] + delta.squeeze(1)
            logits = torch.matmul(H_last, model.lm_head.weight.T)
            next_token_id = torch.argmax(logits, dim=-1, keepdim=True)  # [1, 1]

        record.append(next_token_id)

    # 拼接生成序列（不含 prompt）
    gen_ids = torch.cat(record, dim=-1)  # [1, T]
    record_txt = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    return record_txt


def evaluate_slot_ceval_eos(model, tokenizer, delta, example, prompt, max_len=20, verbose=True):
    """
    基于 generate_by_H_eos 的评估函数，用于 C-Eval 单选题目。

    返回：
    - predict_option: 预测选项，如 'A'
    - is_correct: 是否预测正确
    # """

    # output_text = generate_by_H_eos(model, prompt, tokenizer, delta, answer_len=max_len)
    output_text = generate_by_H_eos_fast(model, prompt, tokenizer, delta, answer_len=max_len)

    if verbose:
        print("🔍 模型生成结果:\n", output_text)

    predict_option = None
    for option in ['A', 'B', 'C', 'D']:
        if option in output_text:
            predict_option = option
            break

    is_correct = (predict_option == example['answer'])
    # return predict_option, is_correct
    return output_text, predict_option, example['answer'], is_correct


def eval_dataset(dataset_name, step=3, max_len=50, lr=1e-2):
    # dataset_name = "computer_network"
    dataset = load_dataset(r"./ceval-exam", name=dataset_name)

    correct = 0
    total = 0

    answer_sheet = []
    for ex in tqdm(dataset['val']):
        # === 构造每道题的 Prompt ===
        prompt = f"""以下是一道单项选择题，请你阅读题目，选择最合适的选项。
        题目：{ex['question']}
        选项：
        A. {ex['A']}
        B. {ex['B']}
        C. {ex['C']}
        D. {ex['D']}
        答案是："""
        prompt = f"""请你扮演一位专业的考试助手，阅读下面的单项选择题，并根据内容在四个选项中选出最合适的一个。
        
        题目：
        {ex['question']}
        
        选项：
        A. {ex['A']}
        B. {ex['B']}
        C. {ex['C']}
        D. {ex['D']}
        
        答案是："""

        #         prompt = f"""请阅读以下单项选择题，并从四个选项中选出一个最合适的答案。
        # 
        # 题目：
        # {ex['question']}
        # 
        # 选项：
        # A. {ex['A']}
        # B. {ex['B']}
        # C. {ex['C']}
        # D. {ex['D']}
        # 
        # 请直接回答，答案是："""

        # === 获取 H_state ===
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        H = outputs.hidden_states[-1]

        # === 训练 delta（例如3步）===
        delta = train_delta_from_H(model, tokenizer, prompt, H, step=step, lr=lr)

        # === 推理与评估 ===
        pred_txt, pre_answer, answer, is_correct = evaluate_slot_ceval_eos(model=model, tokenizer=tokenizer,
                                                                           delta=delta,
                                                                           example=ex, max_len=max_len, prompt=prompt,
                                                                           verbose=False)
        correct += int(is_correct)
        total += 1
        answer_sheet.append([prompt, pred_txt, pre_answer, answer, is_correct, dataset_name])
    print(f"🎯 {dataset_name} Accuracy (per-question delta): {correct}/{total} = {correct / total:.2%}")
    return answer_sheet

In [ ]:
model_path.split("/")[-1]

In [ ]:
for step in [12, 15, 18]:
    max_len = 5
    answer_sheet = []
    for dataset_name in tqdm(dataset_names[:]):
        answer_sheet += eval_dataset(dataset_name=dataset_name, step=step, max_len=max_len, lr=1e-2)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/0_6B/answer_step_{step}.csv", index=False)
pd.DataFrame(answer_sheet)

🎯 college_economics Accuracy (per-question delta): 29/55 = 52.73%


  0%|          | 0/19 [00:00<?, ?it/s]

🎯 college_physics Accuracy (per-question delta): 11/19 = 57.89%


  0%|          | 0/37 [00:00<?, ?it/s]

🎯 college_programming Accuracy (per-question delta): 16/37 = 43.24%


  0%|          | 0/21 [00:00<?, ?it/s]

🎯 computer_architecture Accuracy (per-question delta): 8/21 = 38.10%


  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
for step in [3, 0, 6, 9]:
    max_len = 5
    answer_sheet = []
    for dataset_name in tqdm(dataset_names[:]):
        answer_sheet += eval_dataset(dataset_name=dataset_name, step=step, max_len=max_len, lr=1e-2)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/0_6B/answer_step_{step}.csv", index=False)
pd.DataFrame(answer_sheet)

In [ ]:
for step in [3, 0, 6, 9]:
    max_len = 5
    answer_sheet = []
    for dataset_name in tqdm(dataset_names[:]):
        answer_sheet += eval_dataset(dataset_name=dataset_name, step=step, max_len=max_len, lr=1e-2)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/1_7B/answer_step_{step}.csv", index=False)
pd.DataFrame(answer_sheet)

In [ ]:
for step in [15]:
    max_len = 5
    answer_sheet = []
    for dataset_name in tqdm(dataset_names[:]):
        answer_sheet += eval_dataset(dataset_name=dataset_name, step=step, max_len=max_len, lr=1e-2)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/1_7B/answer_step_{step}.csv", index=False)
pd.DataFrame(answer_sheet)

In [ ]:
for step in [3, 0, 6, 12]:
    max_len = 30
    answer_sheet = []
    for i in tqdm(dataset_names[:]):
        answer_sheet += eval_dataset(i, step=step, max_len=max_len, lr=1e-3)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/1_7B/answer_step_{step}.csv", index=False)

In [ ]:
for step in [3]:
    max_len = 50
    answer_sheet = []
    for i in tqdm(dataset_names[8]):
        answer_sheet += eval_dataset(i, step=step, max_len=max_len, lr=1e-3)
        df_answer = pd.DataFrame(answer_sheet)
        df_answer.to_csv(f"./eval_result/1_7B/answer_step_{step}.csv", index=False)

In [ ]:
pd.DataFrame(answer_sheet)